### This for the Double Integrator

In [1]:
import cupy as cp
import numpy as np

import sys
from os.path import abspath, join
sys.path.append(abspath(join('..')))
from BRATVisualization.rcbrt_visu import RCBRTVisualizer
sys.path.append(abspath(join('../..')))
from LevelSetPy.Utilities import *
from LevelSetPy.Grids import createGrid
from LevelSetPy.Helper import postTimeStepTTR
from LevelSetPy.Visualization import implicit_mesh
from LevelSetPy.DynamicalSystems import DoubleIntegrator
from LevelSetPy.SpatialDerivative import upwindFirstENO2
from LevelSetPy.ExplicitIntegration import artificialDissipationGLF
from LevelSetPy.ExplicitIntegration.Integration import odeCFL2, odeCFLset
from LevelSetPy.ExplicitIntegration.Term import termRestrictUpdate, termLaxFriedrichs

In [7]:
gmin = np.array(([[-1, -1]]),dtype=np.float64).T
gmax = np.array(([[1, 1]]),dtype=np.float64).T
g = createGrid(gmin, gmax, 101, None)

eps_targ = 1.0
u_bound = 1
target_rad = .2 #eps_targ * np.max(g.dx)
dint = DoubleIntegrator(g, u_bound)
value_func = dint.mttr() - target_rad
value_func = np.maximum(0, value_func)

#turn the state space over to the gpu
g.xs = [cp.asarray(x) for x in g.xs]

# Wrap the true Hamiltonian inside the term approximation restriction routine.
finite_diff_data = Bundle(dict(innerFunc = termLaxFriedrichs,
                                innerData = Bundle({'grid':g, 'hamFunc': dint.hamiltonian,
                                                'partialFunc': dint.dynamics,
                                                'dissFunc': artificialDissipationGLF,
                                                'derivFunc': upwindFirstENO2,
                                                }),
                                positive = False,  # direction to grow the updated level set
                            ))
small = 100*eps
t_span = np.linspace(0, 2.0, 20)
options = Bundle(dict(factorCFL=0.75, stats='on', maxStep=realmax, \
                        singleStep='on', postTimestep=postTimeStepTTR))

y = value_func.flatten()
y, finite_diff_data = postTimeStepTTR(0, y, finite_diff_data)
value_func = y.reshape(g.shape)

In [3]:
gmin = np.array(([[-1, -1]]),dtype=np.float64).T
gmax = np.array(([[1, 1]]),dtype=np.float64).T
g = createGrid(gmin, gmax, 101, None)


import scipy.linalg as LA 
eps_targ = 1.0
u_bound = 1
target_rad = .2 #eps_targ * np.max(g.dx)
dint = DoubleIntegrator(g, u_bound)
LA.norm(dint.mttr(), 2)

172.15507974569923

In [135]:
import scipy.io as sio
import scipy.linalg as LA 

class DoubleIntegrator():
    def __init__(self, grid, u_bound=1):
        """
            The base function for the double integrator's 
            minimum time to reach problem.

            Dynamics: \ddot{x}= u,  u \in [-1,1]

            This can represent a car with position 
            x \in \mathbb{R} and with bounded acceleration u acting 
            as the control (negative acceleration corresponds to braking). 
            Let us study the problem of ``parking" the car at the origin, 
            i.e., bringing it to rest at $ x=0$ , in minimal time. 
            It is clear that the system can indeed be brought to rest  
            at the origin from every initial condition. However, since the 
            control is bounded, we cannot do this arbitrarily fast (we are 
            ignoring the trivial case when the system is initialized at the 
            origin). Thus we expect that there exists an optimal control 
            u^* which achieves the transfer in the smallest amount of time. 

            Ref: http://liberzon.csl.illinois.edu/teaching/cvoc/node85.html

            Parameters
            ==========
                grid: an np.meshgrid state space on which we are resolving this integrator dynamics.
        """        
        self.grid     = grid

        matFile = sio.loadmat('../../LevelSetMat/gxs.mat')
        mxs = matFile['xs']; 
        self.grid = grid
        self.grid.xs[0] = mxs[0][0] 
        self.grid.xs[1] = mxs[1][0]

        self.control_law = u_bound
        self.Gamma = self.switching_curve # switching curve

    @property
    def switching_curve(self):
        """
            \Gamma = -(1/2) . x_2 . |x_2|
        """
        self.Gamma = -.5*self.grid.xs[1]*np.abs(self.grid.xs[1])

        return self.Gamma
 
    def hamiltonian(self, t, data, grid_derivs, schemedata): 
        """
            H = \dot{x1} . x2 + \dot{x2} . u + x_0

            Here, x_0 is initial state which is zero.

            Parameters
            ==========
                grid_derivs: Spatial derivatives (finite difference) of 
                            grid points computed with upwinding.
        """
        
        return -(grid_derivs[0]*self.grid.xs[1]+grid_derivs[1]*self.control_law)

    def dynamics(self, t, data, derivMin, derivMax, \
                      schemeData, dim):
        """
            Parameters
            ==========
                dim: The dimension of the ode to return.
        """
        x_dot = [
                    np.abs(self.grid.xs[1]),
                    np.abs(self.control_law * np.zeros_like(self.grid.xs[1]))
        ]

        return x_dot[dim]

    def mttr(self):
        """
            Computes the minimum time we need to reach the 
            switching curve:

            x2 + (sqrt(4x_1 + 2 x_2^2).(x_1 > \Gamma)) + 
            (-x_2 + sqrt(2x_2^2 - 4 x_1) . (x_1 < \Gamma) +
            (|x_2| . (x_1 == \Gamma)).
        """

        #be sure to update the switching curve first
        self.switching_curve
        
        #  Compute the current state on or outside of the 
        # switching curve.

        above_curve = (self.grid.xs[0]>self.Gamma)
        below_curve = (self.grid.xs[0]<self.Gamma)
        on_curve    = (self.grid.xs[0]==self.Gamma)

        print(np.count_nonzero(self.grid.xs[0]>self.Gamma),\
            np.count_nonzero(self.grid.xs[0]<self.Gamma), \
                np.count_nonzero(self.grid.xs[0]==self.Gamma))

        reach_term1  = (self.grid.xs[1] + np.emath.sqrt(4*self.grid.xs[0] + \
                         2 * self.grid.xs[1]**2))*above_curve
        reach_term2 =  (-self.grid.xs[1]+np.emath.sqrt(-4*self.grid.xs[0] + \
                        2 * self.grid.xs[1]**2) )*below_curve
        reach_term3 = np.abs(self.grid.xs[1]) * on_curve
        print(LA.norm(reach_term1, 2))
        print(LA.norm(reach_term2, 2))
        print(LA.norm(reach_term3, 2))
        
        reach_time = reach_term1 + reach_term2 + reach_term3
                      
        return reach_time
           

In [137]:
gmin = np.array(([[-1, -1]]),dtype=np.float64).T
gmax = np.array(([[1, 1]]),dtype=np.float64).T
gtemp = createGrid(gmin, gmax, 101, None)

DI = DoubleIntegrator(gtemp, u_bound)
mttr = DI.mttr()

5099 5099 3
124.73031068905173
124.73031068905173
1.0


In [141]:
LA.norm(mttr, 2)

172.155079745927

In [129]:
t1, t2 = (4*DI.grid.xs[0]), (2 * DI.grid.xs[1]**2)
LA.norm(t1, 2), LA.norm(t2, 2), LA.norm(t1+t2, 2)

(235.57045655175014, 92.13786654356612, 246.01612124314477)

In [8]:
np.maximum(0, [[-1, 2, 3, -5], [-4, 5, -6, 30]])

array([[ 0,  2,  3,  0],
       [ 0,  5,  0, 30]])